In [1]:
import pandas as pd
from datetime import datetime, timedelta, time
import numpy as np
import sys
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.ndimage.interpolation import shift

import time

#this code comes largely from https://github.com/mkduchak/benson-project

In [2]:
# Define list of weeks we want to pull from the MTA portal

def datelist(startdate, num_weeks):
    """
    For a given Saturday, make a list of dates for the n previous Saturdays
    """
    week_list = [startdate + ((timedelta(days=-7))*i) for i in range(num_weeks)]
    clean_weeks = [i.strftime('%y%m%d') for i in week_list]
    return clean_weeks


# Define the last Saturday we're interested in for 2016 and 2017
last18 = datetime(2018, 12, 29)
last17 = datetime(2017, 12, 30)
last16 = datetime(2016, 12, 31)
last15 = datetime(2015, 12, 26)

# We'll import data for the 14 weeks preceeding July 1st for both 2016 and 2017
weeks_to_import = datelist(last16, 52) # + datelist(start16)
#weeks_to_import

In [3]:
turnstile_station_table = pd.read_excel('../data/Turnstile/Remote-Booth-Station.xls').rename({'Remote':'UNIT', 'Booth':'C/A', 'Station':'STATION', 'Line Name': 'LINENAME', 'Division':'DIVISION'}, axis=1)

In [124]:
turnstile_station_table.shape

(768, 5)

In [18]:
def loadturndata(date):
    # Build the filename
    strdate = str(date)
        
    if strdate == "111217":
        #this week is messed up for some reason
        strdate = '111219'
        
    if strdate == '100501':
        #last week is not a full weeek
        strdate = '100505'
    
    filename = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_'+strdate+'.txt'
    
    if strdate == '120714':
        #this week has weird errors so we're skipping those
        df = pd.read_csv(filename, skiprows=10)
    else:  
        # Read in the csv
        df = pd.read_csv(filename)

    return df

def loadoldturndata(date):
    df = loadturndata(date)
    newdf = []
    newdf_cols = ['C/A','UNIT','SCP', 'DATE','TIME','DESC','ENTRIES','EXITS']
    
    for index, row in df.iterrows():
        #construct a row of the new format for each of the 4 hour periods in the old df
        for i in range(0, 8):
            newdf.append([row[0], row[1], row[2], row[3+i*5], row[4+i*5], row[5+i*5], row[6+i*5], row[7+i*5]])
            
    res = pd.DataFrame(data = newdf, columns = newdf_cols).dropna().merge(turnstile_station_table, how = 'left', on= ['UNIT', 'C/A'])
    return res

def loadturnlist(dates):
    """
    We'll iterarte through the list of weeks to create dataframes using loadturndata and then concat together into one dataframe 
    """
    data = pd.DataFrame()
    x = []
    for i in dates:
        print('week '+str(i))
        if str(i) == '100529':
        #this week is missing we'll skip this week
            continue
        
        #before 10/18/2014, the data format was different so we'll have to do some preprocessing
        if int(i) < 141018:
            df = (loadoldturndata(i))
        else:
            df = (loadturndata(i))
            df = df.rename({df.columns[10]: 'EXITS'}, axis=1)
        x.append(df)
        print('done')
    data = pd.concat(x)
    return data

def groups_dict(groups):
        group_dict = defaultdict(int)
        for i in enumerate(list(groups)):
            group_dict[i[1]]= i[0]

        return group_dict
    
def find_diff_prev_row(df_series_col):
        col_array = np.array(df_series_col)
        col_array_shifted = shift(col_array, 1, cval=np.NaN)
        col_diff = abs(col_array - col_array_shifted)

        return col_diff

def find_first_rows_groups(df_series_col):
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    first_row_mask = col_array != col_array_shifted

    return first_row_mask
    
def find_outliers(df_series, multiple_IQR):
    """
    For a series of numerical values, remove the zeros and identify the upper outliers 
    to return a mask for all outliers in series
    """
    non_zeros = df_series.replace(0, None)
    
    adjusted_IQR = (non_zeros.quantile(.75) - non_zeros.quantile(.25)) * multiple_IQR
    outlier_lim = non_zeros.quantile(.75) + adjusted_IQR
    print(outlier_lim)
    
    outliers = [True if x > outlier_lim else False for x in df_series]
    
    outlier_count = sum(outliers)
    all_data_count = len(df_series)
    print('{} outliers identified: {} of all data'.format(outlier_count, round(outlier_count/all_data_count,6)))
    
    return outliers

def find_first_rows_groups(df_series_col):
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    first_row_mask = col_array != col_array_shifted

    return first_row_mask

In [105]:
week = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_100505.txt')
newdf = []
newdf_cols = ['C/A','UNIT','SCP', 'DATE','TIME','DESC','ENTRIES','EXITS']

for index, row in week.iterrows():
    #construct a row of the new format for each of the 4 hour periods in the old df
    for i in range(0, 8):
        newdf.append([row[0], row[1], row[2], row[3+i*5], row[4+i*5], row[5+i*5], row[6+i*5], row[7+i*5]])

In [116]:
ndf = pd.DataFrame(data = newdf, columns = newdf_cols).dropna()

In [131]:
len(ndf) - len(ndf.merge(turnstile_station_table, how='left', on=['C/A', 'UNIT']).dropna())

322

In [137]:
ndf['C/A'].isin(turnstile_station_table['C/A']).value_counts()

True     236685
False       322
Name: C/A, dtype: int64

In [94]:
start_time = time.time()

#before 10/18/2014, the data format was different so we'll have to do something different
last14 = datetime(2014, 12, 27)
last13 = datetime(2013, 12, 28)
last12 = datetime(2012, 12, 29)
last11 = datetime(2011, 12, 31)

last10 = datetime(2010, 12, 25)  #2010 is the last year for which this data exists and it only goes back to the week of may 05 2010

years = [last14]

for year in years:
    print('Downloading Year: '+str(year.year))
    # Note: This takes a long time to run - go treat yourself to a cup of tea!
    weeks_to_import = datelist(year, 35)
    #raw = loadturnlist(weeks_to_import)
    print('saving')
    # Pickle the raw data in case things go south in the cleaning process and you need to start over from here
    #raw.to_pickle('Turnstile/raw_{}_turnstile_data.pkl'.format(year.year))
    print('done')
    raw = pd.read_pickle('../data/Turnstile/raw_{}_turnstile_data.pkl'.format(year.year))
    
    df = raw.rename(columns=lambda x: x.strip().lower())

    print('creating datetime')
    # Concat date and time and convert to datetime object
    df['datetime'] = df['date'] + ' ' + df['time']
    
    clean = []
    for x in df['datetime']:
        try:
            clean.append(datetime.strptime(x, '%m/%d/%Y %H:%M:%S'))
        except:
            clean.append(datetime.strptime(x, '%m-%d-%y %H:%M:%S'))
            
    df['datetime_clean'] = clean
    
    print('done')
    
    print("adding useful columns")
    # Add some helpful date-part columns
    df['year'] = [x.year for x in df['datetime_clean']]
    
    # Create group ID for distinct turnstiles
    df['group'] = df['c/a'].astype(str) + \
                    df['unit'].astype(str) + \
                    df['scp'].astype(str) + \
                    df['station'].astype(str)  + \
                    df['linename'].astype(str) + \
                    df['division'].astype(str) + \
                    df['year'].astype(str)

    
    # Map 'group' string to integer id     
    groups = set(df['group'])

    
    group_id_dict = groups_dict(groups)

    df['group_id'] = [group_id_dict[x] for x in df['group']]

    # Create station ID for distinct stations
    df['station_line'] = df['station'].astype(str) + \
                    df['linename'].astype(str)
    
    print('done')
    print('sorting')
    # Sort values by group id and date to find diff in turnstile counts from prev row 
    #necessary since data is actually a counter that goes up from the installation of the turnstile and is reported every 4 hours
    df.sort_values(['group_id','datetime_clean'], inplace=True)
    df.reset_index(drop=True)
    print('done')
    
    df['entries'] = pd.to_numeric(df['entries'], errors = 'coerce')
    df['exits'] = pd.to_numeric(df['exits'], errors = 'coerce')

    #next we'll delete null values from the dataset
    print('Null entry diffs', df.entries.isnull().sum())
    print('Null exit diffs', df.exits.isnull().sum())
    df.dropna(subset = ['entries', 'exits'], how='any', inplace=True)
    print('Clean Data len:', len(df))
        
    print('finding differences')
    df['entries_diff'] = find_diff_prev_row(pd.to_numeric(df['entries'], errors = 'coerce'))
    df['exit_diff'] = find_diff_prev_row(pd.to_numeric(df['exits'], errors = 'coerce'))
    
    print('done')
    
    print('cleaning invalid data')
    # Identify first rows for each group partition to use as mask when setting invalid values to nan
    #first rows can't be used because that data is not able to be difference
    df['first_row_group'] = find_first_rows_groups(df['group_id'])

    # Make entries_diff and exit_diff nan when first row in group or negative value
    df.loc[df['first_row_group'], 'entries_diff'] = None
    df.loc[df['entries_diff'] < 0, 'entries_diff'] = None

    df.loc[df['first_row_group'], 'exit_diff'] = None
    df.loc[df['exit_diff'] < 0, 'exit_diff'] = None
    print('done')
    
    #here we use 20 times the IQR as outliers, this should catch rollovers and other data artifacts
    #seems like entries of about 5,000 in a four hour period for one unit at one station is a reasonable upper limit
    print('All Data Len:', len(df))
    print('Entries Outliers')
    df['entries_outlier'] = find_outliers(df['entries_diff'], 20)

    #delete these outliers detected here
    clean_df = df.loc[(~df['entries_outlier'])].copy()

    print('\n Exit Outliers')
    clean_df['exit_outlier'] = find_outliers(clean_df['exit_diff'], 20)


    #delete these outliers detected here
    clean_df = clean_df.loc[(~clean_df['exit_outlier'])].copy()

    print('Excluding Outliers Len:', len(clean_df))

    print('Keeping', round(len(clean_df)/len(df), 6))
    
    # Identify first rows for each group partition to use as mask when setting invalid values to nan
    #first rows can't be used because that data is not able to be differenced
    df['first_row_group'] = find_first_rows_groups(df['group_id'])

    # Make entries_diff and exit_diff nan when first row in group or negative value
    df.loc[df['first_row_group'], 'entries_diff'] = None
    df.loc[df['entries_diff'] < 0, 'entries_diff'] = None

    df.loc[df['first_row_group'], 'exit_diff'] = None
    df.loc[df['exit_diff'] < 0, 'exit_diff'] = None
    
    #next we'll delete null values from the dataset
    print('Null entry diffs', clean_df.entries_diff.isnull().sum())
    print('Null exit diffs', clean_df.exit_diff.isnull().sum())
    clean_df.dropna(subset = ['entries_diff', 'exit_diff'], how='any', inplace=True)
    print('Clean Data len:', len(clean_df))
    
    #report total number of data points deleted
    thrown_away = len(df) - len(clean_df)
    print("We're throwing away {} data points - about {} of the total".format(thrown_away, round(thrown_away/len(df), 4)))
    
    #consolidate identitcal datetime columns
    clean_df = clean_df.drop(['datetime', 'date', 'time', 'group'], axis=1)
    clean_df = clean_df.rename({'datetime_clean':'datetime'}, axis=1)
    
    #get rid of useless columns
    clean_df = clean_df.drop(['first_row_group', 'entries_outlier', 'exit_outlier', 'entries', 'exits'], axis=1)
    clean_df = clean_df.rename({'entries_diff':'entries', 'exit_diff':'exits'} , axis=1)
    
    #Now sum up all the turnstiles at each subway station since thats the granularity we desire
    stations = clean_df.groupby(['station_line', 'datetime']).agg({'entries':'sum', 'exits':'sum'}).reset_index()
    #add back the other information from the original df
    stations_clean = stations.merge(clean_df[['station_line', 'datetime','station', 'linename', 'division', 'year']], how='left', copy=False, on=['station_line', 'datetime']).drop_duplicates(subset=['station_line', 'datetime'])
    
    stations_clean.to_pickle('../data/Turnstile/cleaned_{}_turnstile_data.pkl'.format(year.year))
    
    print("--- {} minutes ---" .format((time.time() - start_time)/60))

saving
done
creating datetime
done
adding useful columns
done
sorting
done
Null entry diffs 1
Null exit diffs 1
Clean Data len: 10793646
finding differences
done
cleaning invalid data
done
All Data Len: 10793646
Entries Outliers
4667.0
4228 outliers identified: 0.000392 of all data

 Exit Outliers
3175.0
1788 outliers identified: 0.000166 of all data
Excluding Outliers Len: 10787630
Keeping 0.999443
Null entry diffs 9418
Null exit diffs 9418
Clean Data len: 10778212
We're throwing away 15434 data points - about 0.0014 of the total
--- 6.82455716530482 minutes ---


In [24]:
num = pd.to_numeric(df['exits'], errors = 'coerce')


0.9999999092106849

In [101]:
len(raw) - len(raw.dropna())

162565

In [ ]:
# Rename columns
raw = pd.read_pickle('../data/Turnstile/raw_{}_turnstile_data.pkl'.format(year.year))
df = raw.rename(columns=lambda x: x.strip().lower())

In [ ]:
# Concat date and time and convert to datetime object
df['datetime'] = df['date'] + ' ' + df['time']
clean = []
for x in df['datetime']:
    try:
        clean.append(datetime.strptime(x, '%m/%d/%Y %H:%M:%S'))
    except:
        clean.append(datetime.strptime(x, '%m-%d-%y %H:%M:%S'))
        

In [37]:
df['datetime_clean'] = clean
df['year'] = [x.year for x in df['datetime_clean']]
 

In [38]:
df.head()

c/a  unit       scp      date      time     desc    entries        exits  \
0  A002  R051  02-00-00  12-23-12  11:00:00  REGULAR  3923303.0  1.35307e+06   
1  A002  R051  02-00-00  12-23-12  15:00:00  REGULAR  3923497.0  1.35313e+06   
2  A002  R051  02-00-00  12-23-12  19:00:00  REGULAR  3923744.0  1.35317e+06   
3  A002  R051  02-00-00  12-23-12  23:00:00  REGULAR  3923935.0   1.3532e+06   
4  A002  R051  02-00-00  12-24-12  03:00:00  REGULAR  3923955.0   1.3532e+06   

         station linename division           datetime      datetime_clean  \
0  LEXINGTON AVE   456NQR      BMT  12-23-12 11:00:00 2012-12-23 11:00:00   
1  LEXINGTON AVE   456NQR      BMT  12-23-12 15:00:00 2012-12-23 15:00:00   
2  LEXINGTON AVE   456NQR      BMT  12-23-12 19:00:00 2012-12-23 19:00:00   
3  LEXINGTON AVE   456NQR      BMT  12-23-12 23:00:00 2012-12-23 23:00:00   
4  LEXINGTON AVE   456NQR      BMT  12-24-12 03:00:00 2012-12-24 03:00:00   

   year  
0  2012  
1  2012  
2  2012  
3  2012  
4  2012

In [39]:
# Create group ID for distinct turnstiles
df['group'] = df['c/a'].astype(str) + \
                df['unit'].astype(str) + \
                df['scp'].astype(str) + \
                df['station'].astype(str)  + \
                df['linename'].astype(str) + \
                df['division'].astype(str) + \
                df['year'].astype(str)
                
# Map 'group' string to integer id     
groups = set(df['group'])


def groups_dict(groups):
    group_dict = defaultdict(int)
    for i in enumerate(list(groups)):
        group_dict[i[1]]= i[0]

    return group_dict

group_id_dict = groups_dict(groups)

df['group_id'] = [group_id_dict[x] for x in df['group']]

# Create station ID for distinct stations
df['station_line'] = df['station'].astype(str) + \
                df['linename'].astype(str)

In [40]:
# Sort values by group id and date to find diff in turnstile counts from prev row 
#necessary since data is actually a counter that goes up from the installation of the turnstile and is reported every 4 hours
df.sort_values(['group_id','datetime_clean'], inplace=True)
df.reset_index(drop=True)

def find_diff_prev_row(df_series_col):
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    col_diff = abs(col_array - col_array_shifted)

    return col_diff

In [46]:
df['entries'] = pd.to_numeric(df['entries'], errors = 'coerce')
df['exits'] = pd.to_numeric(df['exits'], errors = 'coerce')

#next we'll delete null values from the dataset
print('Null entry diffs', df.entries.isnull().sum())
print('Null exit diffs', df.exits.isnull().sum())
df.dropna(subset = ['entries', 'exits'], how='any', inplace=True)
print('Clean Data len:', len(df))

Null entry diffs 0
Null exit diffs 1
Clean Data len: 11014511


In [47]:
df['entries_diff'] = find_diff_prev_row(df['entries'])
df['exit_diff'] = find_diff_prev_row(df['exits'])

In [48]:
# Identify first rows for each group partition to use as mask when setting invalid values to nan
#first rows can't be used because that data is not able to be differenced
def find_first_rows_groups(df_series_col):
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    first_row_mask = col_array != col_array_shifted

    return first_row_mask


df['first_row_group'] = find_first_rows_groups(df['group_id'])

# Make entries_diff and exit_diff nan when first row in group or negative value
df.loc[df['first_row_group'], 'entries_diff'] = None
df.loc[df['entries_diff'] < 0, 'entries_diff'] = None

df.loc[df['first_row_group'], 'exit_diff'] = None
df.loc[df['exit_diff'] < 0, 'exit_diff'] = None

In [49]:
#notice that while the majority of data is grouped from 0 to 300, there are a few entries that reach 10^9. 
#These are probably errors from the turnstiles
df.describe()

entries         exits          year      group_id  entries_diff  \
count  1.101451e+07  1.101451e+07  1.101451e+07  1.101451e+07  1.100550e+07   
mean   3.206010e+06  2.629980e+06  2.011997e+03  4.551358e+03  1.207441e+03   
std    7.245853e+06  9.025329e+06  5.074288e-02  2.594401e+03  6.007165e+05   
min    0.000000e+00 -8.789972e+08  2.011000e+03  0.000000e+00  0.000000e+00   
25%    3.431185e+05  2.087390e+05  2.012000e+03  2.305000e+03  3.000000e+00   
50%    1.903791e+06  1.235871e+06  2.012000e+03  4.632000e+03  5.500000e+01   
75%    4.688051e+06  3.555884e+06  2.012000e+03  6.768000e+03  2.120000e+02   
max    9.060372e+08  9.276785e+08  2.012000e+03  9.015000e+03  9.024560e+08   

          exit_diff  
count  1.100550e+07  
mean   1.211696e+03  
std    6.041444e+05  
min    0.000000e+00  
25%    2.000000e+00  
50%    4.000000e+01  
75%    1.510000e+02  
max    9.262858e+08

In [50]:
def find_outliers(df_series, multiple_IQR):
    """
    For a series of numerical values, remove the zeros and identify the upper outliers 
    to return a mask for all outliers in series
    """
    non_zeros = df_series.replace(0, None)
    
    adjusted_IQR = (non_zeros.quantile(.75) - non_zeros.quantile(.25)) * multiple_IQR
    outlier_lim = non_zeros.quantile(.75) + adjusted_IQR
    print(outlier_lim)
    
    outliers = [True if x > outlier_lim else False for x in df_series]
    
    outlier_count = sum(outliers)
    all_data_count = len(df_series)
    print('{} outliers identified: {} of all data'.format(outlier_count, round(outlier_count/all_data_count,6)))
    
    return outliers

In [51]:
#here we use 20 times the IQR as outliers, this should catch rollovers and other data artifacts
#seems like entries of about 5,000 in a four hour period for one unit at one station is a reasonable upper limit
print('All Data Len:', len(df))
print('Entries Outliers')
df['entries_outlier'] = find_outliers(df['entries_diff'], 20)

#delete these outliers detected here
clean_df = df.loc[(~df['entries_outlier'])].copy()

print('\n Exit Outliers')
clean_df['exit_outlier'] = find_outliers(clean_df['exit_diff'], 20)


#delete these outliers detected here
clean_df = clean_df.loc[(~clean_df['exit_outlier'])].copy()

print('Excluding Outliers Len:', len(clean_df))

print('Keeping', round(len(clean_df)/len(df), 6))

All Data Len: 11014511
Entries Outliers
4393.0
3061 outliers identified: 0.000278 of all data


MemoryError: Unable to allocate array with shape (12, 11011450) and data type object

MemoryError: Unable to allocate array with shape (2, 11014511) and data type float64

MemoryError: Unable to allocate array with shape (8, 11014511) and data type object

In [ ]:
#next we'll delete null values from the dataset
print('Null entry diffs', clean_df.entries_diff.isnull().sum())
print('Null exit diffs', clean_df.exit_diff.isnull().sum())
clean_df.dropna(subset = ['entries_diff', 'exit_diff'], how='any', inplace=True)
print('Clean Data len:', len(clean_df))

In [ ]:
#report total number of data points deleted
thrown_away = len(df) - len(clean_df)
print("We're throwing away {} data points - about {} of the total".format(thrown_away, round(thrown_away/len(df), 4)))

In [70]:
#consolidate identitcal datetime columns
clean_df = clean_df.drop(['datetime', 'date', 'time', 'group'], axis=1)
clean_df = clean_df.rename({'datetime_clean':'datetime'}, axis=1)

In [71]:
#get rid of useless columns
clean_df = clean_df.drop(['first_row_group', 'entries_outlier', 'exit_outlier', 'entries', 'exits'], axis=1)
clean_df = clean_df.rename({'entries_diff':'entries', 'exit_diff':'exits'} , axis=1)

In [72]:
#Now sum up all the turnstiles at each subway station since thats the granularity we desire
stations = clean_df.groupby(['station_line', 'datetime']).agg({'entries':'sum', 'exits':'sum'}).reset_index()

In [1]:
stations.groupby(['station_line', 'datetime']).count().describe()

NameError: name 'stations' is not defined

In [95]:
#add back the other information from the original df
stations_clean = stations.merge(clean_df[['station_line', 'datetime','station', 'linename', 'division', 'year']], how='left', copy=False, on=['station_line', 'datetime']).drop_duplicates(subset=['station_line', 'datetime'])

In [96]:
len(stations) - len(stations_clean)

0

In [88]:
#stations_clean.to_pickle('Turnstile/cleaned_2014_turnstile_data.pkl')

In [166]:
#read back data if starting from here:
stations_clean = pd.read_pickle('../data/Turnstile/cleaned_2015_turnstile_data.pkl')
#we have to drop a few entries due to there  being mismatches in our data assigning turnstiles to stops 
#this amounts to less than 1% of the data though
print('Bad entries', stations_clean.isnull().sum()/len(stations_clean))
stations_clean = stations_clean.dropna()

Bad entries station_line    0.0
datetime        0.0
entries         0.0
exits           0.0
station         0.0
linename        0.0
division        0.0
year            0.0
dtype: float64


In [163]:
stations_daily = stations_clean[['station_line', 'datetime', 'entries', 'exits']].groupby(['station_line']).resample('d', on = 'datetime').sum().reset_index()

In [164]:
stations_daily_clean = stations_daily.merge(stations_clean[['station_line', 'station','linename', 'division', 'year']].drop_duplicates(), how='left', on=['station_line']).drop_duplicates(subset=['station_line', 'datetime'])

In [165]:
stations_daily_clean.to_pickle('../data/Turnstile/2010_daily_turnstile_data.pkl')

In [140]:
len(stations_daily_clean()/len(stations_daily_clean)

0.9983375329452313

In [142]:
len(stations_clean.dropna())/ len(stations_clean)

0.9918556986826111

In [80]:
stations_daily.merge(stations_clean[['station_line', 'station','linename', 'division', 'year']].drop_duplicates(), how='left', on='station_line').drop_duplicates(subset=['station_line', 'datetime'])

station_line   datetime  entries    exits    station linename division  \
0             1 AVL 2016-01-02  14425.0  16938.0       1 AV        L      BMT   
1             1 AVL 2016-01-03  14437.0  15092.0       1 AV        L      BMT   
2             1 AVL 2016-01-04  21800.0  24519.0       1 AV        L      BMT   
3             1 AVL 2016-01-05  22641.0  24025.0       1 AV        L      BMT   
4             1 AVL 2016-01-06  23787.0  24490.0       1 AV        L      BMT   
...             ...        ...      ...      ...        ...      ...      ...   
173718   ZEREGA AV6 2016-12-26   1200.0   1041.0  ZEREGA AV        6      IRT   
173719   ZEREGA AV6 2016-12-27   2158.0   1895.0  ZEREGA AV        6      IRT   
173720   ZEREGA AV6 2016-12-28   2255.0   2134.0  ZEREGA AV        6      IRT   
173721   ZEREGA AV6 2016-12-29   2179.0   1974.0  ZEREGA AV        6      IRT   
173722   ZEREGA AV6 2016-12-30   2159.0   1875.0  ZEREGA AV        6      IRT   

        year  
0       2016  
1       2016  
2       2016  
3       2016  
4       2016  
...      ...  
173718  2016  
173719  2016  
173720  2016  
173721  2016  
173722  2016  

[171539 rows x 8 columns]

In [213]:
stations_clean = pd.read_pickle('../data/Turnstile/cleaned_2017_turnstile_data.pkl')

In [182]:
stations_clean.station.str.contains('CYPRUS HILLS').sum()

0

In [226]:
stations_clean[(stations_clean.station == '72 ST-2 AVE')]

station_line            datetime  entries   exits      station  \
3246177  72 ST-2 AVEQ 2016-12-31 04:00:00      0.0     0.0  72 ST-2 AVE   
3246194  72 ST-2 AVEQ 2016-12-31 08:00:00      0.0     0.0  72 ST-2 AVE   
3246211  72 ST-2 AVEQ 2016-12-31 12:00:00      2.0     1.0  72 ST-2 AVE   
3246228  72 ST-2 AVEQ 2016-12-31 16:00:00      0.0     0.0  72 ST-2 AVE   
3246245  72 ST-2 AVEQ 2016-12-31 20:00:00      1.0     0.0  72 ST-2 AVE   
...               ...                 ...      ...     ...          ...   
3283397  72 ST-2 AVEQ 2017-12-29 04:00:00    235.0   259.0  72 ST-2 AVE   
3283414  72 ST-2 AVEQ 2017-12-29 08:00:00    816.0  2721.0  72 ST-2 AVE   
3283431  72 ST-2 AVEQ 2017-12-29 12:00:00   4135.0  5783.0  72 ST-2 AVE   
3283448  72 ST-2 AVEQ 2017-12-29 16:00:00   6546.0  4056.0  72 ST-2 AVE   
3283465  72 ST-2 AVEQ 2017-12-29 20:00:00   3866.0  3391.0  72 ST-2 AVE   

        linename division  year  
3246177        Q      IND  2016  
3246194        Q      IND  2016  
3246211        Q      IND  2016  
3246228        Q      IND  2016  
3246245        Q      IND  2016  
...          ...      ...   ...  
3283397        Q      IND  2017  
3283414        Q      IND  2017  
3283431        Q      IND  2017  
3283448        Q      IND  2017  
3283465        Q      IND  2017  

[2217 rows x 8 columns]